<a href="https://colab.research.google.com/github/zenithexpo/Major-Project/blob/master/NER_using_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
cd drive/

/content/drive


In [5]:
cd MyDrive/

/content/drive/MyDrive


In [6]:
cd ner/

/content/drive/MyDrive/ner


In [7]:
cd ner_tensorflow/

/content/drive/MyDrive/ner/ner_tensorflow


In [5]:
!mkdir data
!mkdir models

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘models’: File exists


In [16]:
!curl https://groups.csail.mit.edu/sls/downloads/movie/engtest.bio -o data/test.txt
!curl https://groups.csail.mit.edu/sls/downloads/movie/engtrain.bio -o data/train.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  246k  100  246k    0     0   249k      0 --:--:-- --:--:-- --:--:--  249k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  989k  100  989k    0     0  1120k      0 --:--:-- --:--:-- --:--:-- 1119k


In [79]:
!pip install tensorflow==1.13.1

     |████████████████████████████████| 92.6MB 83kB/s 
     |████████████████████████████████| 51kB 4.0MB/s 
     |████████████████████████████████| 368kB 41.4MB/s 
     |████████████████████████████████| 3.2MB 42.5MB/s 
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [1]:
!pip install keras==2.2.4

     |████████████████████████████████| 317kB 4.9MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [2]:
import tensorflow as tf
from functools import partial
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import pickle

params = {
    'dim' : 300,# dimension of embeddings
    'maximum_steps' : 1000,# number of training steps
    'lstm_size' : 150,# dimension of LSTM
    'batch_size' : 25,# batch size
    'max_words' : 10000,# maximum number of words to embed
    'padding_size' : 20,# maximum sentence size
    'num_classes' : 14,# number of unique classes
    'save_dir' : 'models/'# directory to save hash tables, model weights, etc.
}


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [8]:
def save_obj(directory, obj, name):
    '''Helper function using pickle to save and load objects'''
    with open(directory + name + '.pkl', 'wb+') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(directory, name):
    '''Helper function using pickle to save and load objects'''
    with open(directory + name + ".pkl", "rb") as f:
        return pickle.load(f)
   
def load_data(file = "data/train.txt"):
    '''Helper function to load and transform inputs and labels
    included as a separate function due to NER-specific evaluation needs:
        tensorflow does not have multi-class precision/accuracy as a metric
        so data_y is needed to manually calculate evaluations'''
    file = open(file, 'r')
    sentence, labels = [], []
    data_x, data_y = [], []
    for line in file:
        line = line.strip("\n").split("\t")
       
        # lines with len > 1 are words
        if len(line) > 1:
            sentence.append(line[1])
            labels.append(line[0][2:]) if len(line[0]) > 1 else labels.append(line[0])
       
        # lins with len == 1 are sentence breaks
        if len(line) == 1:
            data_x.append(' '.join(sentence))
            data_y.append(labels)
            sentence, labels = [], []
    return data_x, data_y

def make_tokenizer(file = "data/train.txt", params = params):
    ''' In order for one hot encoding of words and labels to work,
    every word and label has to be seen at least once to make a hashing table.
    This function outputs hash tables for the words and the labels
    that can be used to one-hot-encode them in the generator
    '''
    # Load parameters and data
    max_words = params['max_words']
    padding_size = params['padding_size']
    save_dir = params['save_dir']
    data_x, data_y = load_data(file)
           
    # Use the Keras tokenizer API to generate hashing table for data_x
    tokenizer = Tokenizer(num_words = max_words)
   
    tokenizer.fit_on_texts(data_x)
    word_index = tokenizer.word_index
   
    # Flatten data_y and create hashing table using set logic
    data_y_flattened = [item for sublist in data_y for item in sublist]
    data_x_flattened = [item for sublist in data_x for item in sublist]
   
    labels_index = dict([(y, x + 1) for x, y in enumerate(sorted(set(data_y_flattened)))])
    labels = []
    for item in data_y:
        labels.append([labels_index.get(i) for i in item])
    labels_lookup = {v : k for k, v in labels_index.items()} # reverse dictionary for lookup
    # save hash tables to disk for model serving
    for item, name in zip([word_index, labels_index, labels_lookup],
                          ["word_index", "labels_index", "labels_lookup"]):
        save_obj(save_dir, item, name)
    return word_index, labels_index, labels_lookup

word_index, labels_index, labels_lookup = make_tokenizer()

In [9]:
labels_index


{'ACTOR': 1,
 'CHARACTER': 2,
 'DIRECTOR': 3,
 'GENRE': 4,
 'O': 5,
 'PLOT': 6,
 'RATING': 7,
 'RATINGS_AVERAGE': 8,
 'REVIEW': 9,
 'SONG': 10,
 'TITLE': 11,
 'TRAILER': 12,
 'YEAR': 13}

In [10]:
def generate_batches(file = "data/train.txt", params = params, train = True):
    ''' Generate minibatch with dimensions:
    batch_x : (batch_size, max_len)
    lengths : (batch_size,)
    batch_y : (batch_size, num_classes)
   
    file : path to .txt containing training data in BIO format
    '''
   
    batch_size = params['batch_size']
    max_len = params['padding_size']
    save_dir = params['save_dir']
   
    # load hash tables for tokenization
    for item, name in zip([word_index, labels_index, labels_lookup],
                          ["word_index", "labels_index", "labels_lookup"]):
        item = load_obj(save_dir, name)
   
    while True:
        with open(file, 'r') as f:
            batch_x, lengths, batch_y = [], [], []
            words, labels = [], []
            for line in f:
                line = line.strip("\n").split("\t")
                # lines with len > 1 are words
                if len(line) > 1:
                    labels.append(line[0][2:]) if len(line[0]) > 1 else labels.append(line[0])
                    words.append(line[1])

                # lines with len == 1 are breaks between sentences
                if len(line) == 1:
                    words = [word_index.get(x) if x in word_index.keys() else 0 for x in words]
                    labels = [labels_index.get(y) for y in labels]
                    batch_x.append(words)
                    batch_y.append(labels)
                    lengths.append(min(len(words), max_len))
                    words, labels = [], []

                if len(batch_x) == batch_size:
                    batch_x = pad_sequences(batch_x, maxlen = max_len, value = 0, padding = "post")
                    batch_y = pad_sequences(batch_y, maxlen = max_len, value = 0, padding = "post")
                    yield (batch_x, lengths), batch_y
                    batch_x, lengths, batch_y = [], [], []
            if train == False:
                break

In [11]:
# For model training, we need an input function that will feed a tf.Dataset
def input_fn(file, params = None, train = True):
    params = params if params is not None else {}
    shapes = (([None, None], [None]), [None, None]) # batch_x, lengths, batch_y shapes
    types = ((tf.int32, tf.int32), tf.int32)        # batch_x, lengths, batch_y data types
   
    generator = partial(generate_batches, file, train = train)
    dataset = tf.data.Dataset.from_generator(generator, types, shapes)
    return dataset

# For model serving, we need a serving function that will feed tf.placeholders
def serving_input_fn():
    words = tf.placeholder(dtype=tf.int32, shape=[None, None], name='words')
    length = tf.placeholder(dtype=tf.int32, shape=[None], name='length')
    receiver_tensors = {'words': words, 'length': length}
    features = {'words': words, 'length': length}
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)

In [12]:
def model_fn(features, labels, mode, params = params):
    # import the data and unpack the features
    # serving input_fn returns a dict, convert to multivalue obj
    if isinstance(features, dict):
        features = features['words'], features['length']
   
    words, length = features
   
    # Embedding
    embedding = tf.Variable(tf.random_normal([params['max_words'], params['dim']]))
    embedding_lookup_for_x = tf.nn.embedding_lookup(embedding, words)
   
    # LSTM
    lstm_cell_fw = tf.nn.rnn_cell.BasicLSTMCell(params['lstm_size'], state_is_tuple = True)
    lstm_cell_bw = tf.nn.rnn_cell.BasicLSTMCell(params['lstm_size'], state_is_tuple = True)
    states, final_state = tf.nn.bidirectional_dynamic_rnn(
                                        cell_fw = lstm_cell_fw,
                                        cell_bw = lstm_cell_bw,
                                        inputs = embedding_lookup_for_x,
                                        dtype = tf.float32,
                                        time_major = False,
                                        sequence_length = length)
    lstm_out = tf.concat([states[0], states[1]], axis = 2)
       
    # Conditional random fields
    logits = tf.layers.dense(lstm_out, params['num_classes'])
    crf_params = tf.get_variable("crf", [params['num_classes'], params['num_classes']],
                                 dtype=tf.float32)
    pred_ids, _ = tf.contrib.crf.crf_decode(logits, crf_params, length)
    training = (mode == tf.estimator.ModeKeys.TRAIN)
   
    # Prediction
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'pred_ids': pred_ids,
            'tags': words,
            'length' : length,
        }
        export_outputs = {
          'prediction': tf.estimator.export.PredictOutput(predictions)
      }
       
        return tf.estimator.EstimatorSpec(mode, predictions=predictions,
                                          export_outputs=export_outputs)
   
    # Loss functions and optimizers
    log_likelihood, _ = tf.contrib.crf.crf_log_likelihood(
        logits, labels, length, crf_params)
   
    loss = tf.reduce_mean(-log_likelihood)
    train_op = tf.train.AdamOptimizer().minimize(
        loss, global_step = tf.train.get_or_create_global_step())
       
    # Training
    if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode = mode,
                                           loss = loss,
                                           train_op = train_op)

In [13]:
# Spin up the estimator
config = tf.estimator.RunConfig()
estimator = tf.estimator.Estimator(model_fn, 'models/model', config, params)

# Create train spec
train_input_fn = partial(input_fn, "data/train.txt", params = params)
train_spec = tf.estimator.TrainSpec(train_input_fn)

# Create evaluation spec
eval_input_fn = partial(input_fn, "data/test.txt", params = params, train = False)
eval_spec = tf.estimator.EvalSpec(eval_input_fn)

INFO:tensorflow:Using config: {'_model_dir': 'models/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe82cf2d2d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [14]:
import time

In [15]:
ts = time.time()
estimator.train(input_fn = train_input_fn, max_steps = 1000)
te = time.time()
print("Completed in {} seconds".format(int(te - ts)))
estimator.export_savedmodel('models/saved_model/', serving_input_fn)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
INFO:tensorflow:Calling model_fn.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use keras.layers.dense instead.

For

/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into models/model/model.ckpt.
INFO:tensorflow:loss = 25.802078, step = 1
INFO:tensorflow:global_step/sec: 7.91578
INFO:tensorflow:loss = 5.56465, step = 101 (12.636 sec)
INFO:tensorflow:global_step/sec: 7.45503
INFO:tensorflow:loss = 2.5894566, step = 201 (13.413 sec)
INFO:tensorflow:global_step/sec: 6.74372
INFO:tensorflow:loss = 1.5749393, step = 301 (14.828 sec)
INFO:tensorflow:global_step/sec: 7.42633
INFO:tensorflow:loss = 3.9944944, step = 401 (13.467 sec)
INFO:tensorflow:global_step/sec: 8.58904
INFO:tensorflow:loss = 6.255591, step = 501 (11.642 sec)
INFO:tensorflow:global_step/sec: 7.37029
INFO:tensorflow:loss = 1.154845, step = 601 (13.569 sec)
INFO:tensorflow:global_step/sec: 6.81161
INFO:tensorflow:loss = 0.4085395, step = 701 (14.680

b'models/saved_model/1614950330'

In [22]:
import numpy as np
import keras


In [30]:
class Recall(tf.keras.metrics.Recall):

  def __init__(self, *, class_id, **kwargs):
    super().__init__(**kwargs)
    self.class_id= class_id

  def update_state(self, y_true, y_pred, sample_weight=None):
    y_true = y_true[:, self.class_id]
    y_pred = tf.cast(tf.equal(
      tf.math.argmax(y_pred, axis=-1), self.class_id), dtype=tf.float32)
    return super().update_state(y_true, y_pred, sample_weight)

In [41]:
# Generate predictions
predictions = estimator.predict(eval_input_fn)

# Load hash tables and true labels
labels_index = load_obj(params['save_dir'], "labels_index")
_, true = load_data("data/test.txt")

# Specify which label_index is non-entity
dummy_label = labels_index.get("O")

# Convert [[string, string], [string, string] ...] to [[int, int], [int, int]]
# with hashing table for label indexes
labels = []
for row in true:
    labels.append([labels_index.get(y) for y in row])
   
# Loop through preds, labels and calculate metrics
precisions, recalls, f1s = [], [], []
for pred, truee in zip(predictions, labels):
    pred = pred['pred_ids'][:pred['length']] # undo pad_sequences
    pred = [x for x in pred if x != dummy_label] # remove preds that aren't entities
    truee = np.asarray([x for x in true if x != dummy_label])
    print(truee,pred)
    m = tf.keras.metrics.Recall(truee,pred)
    recall = m.result().numpy()
    # recall.update_state(truee,pred)    
    # recall = tf.keras.metrics.Recall(truee, pred)
    recalls.append(recall)
    precision = calc_precision(truee, pred)
    precisions.append(precision)
    f1s.append(calc_f1(precision, recall))
   
print("Precision: {} \nRecall: {} \nF1-score: {}".format(np.around(np.mean(precisions), 3),
                                                         np.around(np.mean(recalls), 3),
                                                         np.around(np.mean(f1s), 3)))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/model/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[list(['O', 'O', 'O', 'O', 'GENRE', 'GENRE', 'O', 'YEAR', 'YEAR'])
 list(['O', 'O', 'O', 'O', 'O', 'PLOT', 'PLOT', 'PLOT'])
 list(['O', 'O', 'RATINGS_AVERAGE', 'RATINGS_AVERAGE', 'O', 'O', 'O', 'ACTOR', 'ACTOR'])
 ... list(['O', 'CHARACTER', 'CHARACTER', 'O'])
 list(['O', 'O', 'O', 'GENRE', 'O', 'O', 'CHARACTER', 'CHARACTER'])
 list(['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'CHARACTER', 'CHARACTER', 'CHARACTER'])] [4, 4]


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


TypeError: ignored

In [42]:
 from pathlib import Path
from tensorflow.contrib import predictor

LINE = 'did george clooney make a science fiction movie in the 1980s'


def predict(line, export_dir = 'models/saved_model/', params = params):
    # Load hash tables
    word_index = load_obj(params['save_dir'], "word_index")
    labels_lookup = load_obj(params['save_dir'], "labels_lookup")
   
    # Identify and load model weights
    subdirs = [x for x in Path(export_dir).iterdir()
                   if x.is_dir() and 'temp' not in str(x)]
    latest_model = str(sorted(subdirs)[-1])
    predict_fn = predictor.from_saved_model(latest_model)
               
    # Preprocess sentence input
    line = line.strip().split()
    vector = [word_index.get(x) if x in word_index.keys() else 0 for x in line] # tokenize
    vector[len(vector):20] = [0] * (20 - len(vector)) # pad prediction
       
    # Calculate precision and transform for display
    predictions = predict_fn({'words': [vector], 'length': [len(line)]})
    tags = predictions.get('tags')
    preds = predictions.get('pred_ids')
    for tag, pred in zip(tags, preds):
        tag = [word for word in tag if word != 0] # unpad
        pred = pred[:len(tag)]
        pred = [labels_lookup.get(num) for num in pred] #untokenize
        print(line, "\n", pred)
   
predict(LINE)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from models/saved_model/1614950330/variables/variables
['did', 'george', 'clooney', 'make', 'a', 'science', 'fiction', 'movie', 'in', 'the', '1980s'] 
 ['O', 'ACTOR', 'ACTOR', 'O', 'O', 'GENRE', 'GENRE', 'O', 'O', 'O', 'YEAR']
